# Regression Exercise - Solutions

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [89]:
import pandas as pd

In [90]:
housing = pd.read_csv('cal_housing_clean.csv')

In [91]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [92]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [93]:
x_data = housing.drop(['medianHouseValue'],axis=1)

In [94]:
y_val = housing['medianHouseValue']

In [95]:
from sklearn.model_selection import train_test_split

In [96]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [97]:
from sklearn.preprocessing import MinMaxScaler

In [98]:
scaler = MinMaxScaler()

In [99]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [100]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [101]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [102]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [103]:
import tensorflow as tf

In [104]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [105]:
feat_cols = [ age,rooms,bedrooms,pop,households,income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [106]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,
                                            shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [107]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sr/9k0jqkfs33q22ppyt_t7yjt9716826/T/tmpbhqmebe6', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10d6830b8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [108]:
model.train(input_fn=input_func,steps=25000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/sr/9k0jqkfs33q22ppyt_t7yjt9716826/T/tmpbhqmebe6/model.ckpt.
INFO:tensorflow:loss = 4.70844e+11, step = 1
INFO:tensorflow:global_step/sec: 633.373
INFO:tensorflow:loss = 9.45479e+11, step = 101 (0.159 sec)
INFO:tensorflow:global_step/sec: 730.107
INFO:tensorflow:loss = 5.96638e+11, step = 201 (0.137 sec)
INFO:tensorflow:global_step/sec: 796.56
INFO:tensorflow:loss = 5.46255e+11, step = 301 (0.126 sec)
INFO:tensorflow:global_step/sec: 797.721
INFO:tensorflow:loss = 9.06621e+11, step = 401 (0.125 sec)
INFO:tensorflow:global_step/sec: 768.026
INFO:tensorflow:loss = 2.61556e+11, step = 501 (0.130 sec)
INFO:tensorflow:global_step/sec: 623.196
INFO:tensorflow:loss = 3.03787e+11, step = 601 (0.160 sec)
INFO:tensorflow:global_step/sec: 589.418
INFO:tensorflow:loss = 3.99056e+11, step = 701 (0.170 sec)
INFO:tensorflow:global_step/sec: 581.757
INFO:tensorflow:loss = 4.01421e+11, step = 801 (0.17

INFO:tensorflow:loss = 8.09601e+10, step = 8001 (0.133 sec)
INFO:tensorflow:global_step/sec: 563.476
INFO:tensorflow:loss = 8.35997e+10, step = 8101 (0.177 sec)
INFO:tensorflow:global_step/sec: 631.05
INFO:tensorflow:loss = 5.90797e+10, step = 8201 (0.160 sec)
INFO:tensorflow:global_step/sec: 626.725
INFO:tensorflow:loss = 4.78233e+10, step = 8301 (0.158 sec)
INFO:tensorflow:global_step/sec: 745.711
INFO:tensorflow:loss = 4.02899e+10, step = 8401 (0.134 sec)
INFO:tensorflow:global_step/sec: 841.405
INFO:tensorflow:loss = 8.63236e+10, step = 8501 (0.119 sec)
INFO:tensorflow:global_step/sec: 793.108
INFO:tensorflow:loss = 1.39572e+11, step = 8601 (0.126 sec)
INFO:tensorflow:global_step/sec: 774.78
INFO:tensorflow:loss = 1.64849e+11, step = 8701 (0.129 sec)
INFO:tensorflow:global_step/sec: 761.689
INFO:tensorflow:loss = 2.95631e+10, step = 8801 (0.132 sec)
INFO:tensorflow:global_step/sec: 681.654
INFO:tensorflow:loss = 6.60554e+10, step = 8901 (0.147 sec)
INFO:tensorflow:global_step/sec: 

INFO:tensorflow:loss = 4.2545e+10, step = 16101 (0.138 sec)
INFO:tensorflow:global_step/sec: 636.578
INFO:tensorflow:loss = 2.99563e+10, step = 16201 (0.158 sec)
INFO:tensorflow:global_step/sec: 603.132
INFO:tensorflow:loss = 1.74897e+11, step = 16301 (0.165 sec)
INFO:tensorflow:global_step/sec: 590.281
INFO:tensorflow:loss = 5.60105e+10, step = 16401 (0.169 sec)
INFO:tensorflow:global_step/sec: 719.848
INFO:tensorflow:loss = 1.32348e+11, step = 16501 (0.139 sec)
INFO:tensorflow:global_step/sec: 852.485
INFO:tensorflow:loss = 5.70093e+10, step = 16601 (0.117 sec)
INFO:tensorflow:global_step/sec: 850.791
INFO:tensorflow:loss = 7.80698e+10, step = 16701 (0.118 sec)
INFO:tensorflow:global_step/sec: 847.939
INFO:tensorflow:loss = 5.89704e+10, step = 16801 (0.118 sec)
INFO:tensorflow:global_step/sec: 770.018
INFO:tensorflow:loss = 5.54862e+10, step = 16901 (0.130 sec)
INFO:tensorflow:global_step/sec: 740.703
INFO:tensorflow:loss = 7.24299e+10, step = 17001 (0.135 sec)
INFO:tensorflow:global

INFO:tensorflow:global_step/sec: 808.806
INFO:tensorflow:loss = 8.94404e+10, step = 24201 (0.124 sec)
INFO:tensorflow:global_step/sec: 794.15
INFO:tensorflow:loss = 1.14481e+11, step = 24301 (0.126 sec)
INFO:tensorflow:global_step/sec: 631.197
INFO:tensorflow:loss = 5.61752e+10, step = 24401 (0.159 sec)
INFO:tensorflow:global_step/sec: 586.218
INFO:tensorflow:loss = 9.47448e+10, step = 24501 (0.170 sec)
INFO:tensorflow:global_step/sec: 636.517
INFO:tensorflow:loss = 4.79966e+10, step = 24601 (0.157 sec)
INFO:tensorflow:global_step/sec: 589.507
INFO:tensorflow:loss = 1.01608e+11, step = 24701 (0.170 sec)
INFO:tensorflow:global_step/sec: 602.54
INFO:tensorflow:loss = 1.19791e+11, step = 24801 (0.165 sec)
INFO:tensorflow:global_step/sec: 722.664
INFO:tensorflow:loss = 4.99066e+10, step = 24901 (0.138 sec)
INFO:tensorflow:Saving checkpoints for 25000 into /var/folders/sr/9k0jqkfs33q22ppyt_t7yjt9716826/T/tmpbhqmebe6/model.ckpt.
INFO:tensorflow:Loss for final step: 1.45864e+11.


** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [109]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [110]:
pred_gen = model.predict(predict_input_func)

In [111]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from /var/folders/sr/9k0jqkfs33q22ppyt_t7yjt9716826/T/tmpbhqmebe6/model.ckpt-25000


** Calculate the RMSE. Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [112]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [113]:
from sklearn.metrics import mean_squared_error

In [114]:
mean_squared_error(y_test,final_preds)**0.5

98321.662413254002

# Great Job!